# Quartely Crypto Market Review Framework

## Setting up the folders and import modules

In [3]:
import os
from datetime import date, datetime
import calendar
import requests
import pandas as pd 
import numpy as np
from threading import Thread 
import sqlite3
import time
import json
Data_Path = 'Data'
Result_Path = 'Results'
for path in [Data_Path,Result_Path]:
    if not os.path.exists(path):
        os.makedirs(path)

## Add new exchanges to exchange csv file

In [9]:
url='https://min-api.cryptocompare.com/data/all/exchanges'
data = requests.get(url).json()
ccc_new_df = pd.DataFrame.from_dict(data).T
ccc_new_df.to_csv(os.path.join(Data_Path,'CCCExchanges_List.csv'))
new_exchanges = set(ccc_new_df.index.values)
ccc_old_df = pd.read_csv(os.path.join(Data_Path,'CCCExchanges_Old.csv'))
ccc_old_df = ccc_old_df[~pd.isnull(ccc_old_df['Country'])]
old_exchanges = set(ccc_old_df['Source'].values)
old_exchanges.add('CCCAGG')

In [10]:
## In old but not in new
if old_exchanges-new_exchanges:
    print("That's weird: ",old_exchanges-new_exchanges)
    
## In new but not in old
print('%s exchanges not included in the old exchange list'%len(new_exchanges-old_exchanges))
new_ex = pd.DataFrame(list(new_exchanges-old_exchanges),columns = ['Source'])
complete_ex = ccc_old_df.append(new_ex.sort_values(by=['Source']))
complete_ex = complete_ex[ccc_old_df.columns]
complete_ex.to_csv(os.path.join(Data_Path,'CCCExchanges.csv'),index=False)
complete_ex

0 exchanges not included in the old exchange list


,Source,Source_small,Country,Flag,Region,Remark,Excluded,Reason for Exclusion,rank
0,ABCC,abcc,Singapore,🇸🇬,East Asia,NaN,False,NaN,1.0
1,ACX,acx,Australia,🇺🇸,Oceania,NaN,False,NaN,2.0
2,AidosMarket,aidosmarket,Unknown,🏴‍,Unknown,NaN,False,NaN,3.0
3,aliexchange,aliexchange,Estonia,????,Eastern Europe,NaN,False,NaN,4.0
4,alphaex,alphaex,Malta,🇲🇹,Western Europe,NaN,False,NaN,5.0
...,...,...,...,...,...,...,...,...,...
234,zloadr,zloadr,UK,🇬🇧,Western Europe,NaN,False,NaN,236.0
235,bitflyereu,bitflyereu,Luxembourg,NaN,Western Europe,NaN,False,NaN,237.0
236,deribit,deribit,Netherlands,????,Western Europe,NaN,False,NaN,238.0
237,binanceus,binanceus,USA,🇺🇸,North America,NaN,False,NaN,239.0


In [21]:
## Download all coins from CCC
def unix_time(d):
    return calendar.timegm(d.timetuple())
end_date = datetime.today()

url='https://min-api.cryptocompare.com/data/all/exchanges'
data = requests.get(url).json()
ccc_df = pd.DataFrame.from_dict(data).T

ex_list = list(ccc_df.index)
ex_list.remove('EtherDelta')
ex_list.append('EtherDelta')

conn = sqlite3.connect(os.path.join(Data_Path,"CCC.db"))

cursor = conn.cursor()

all_crypto = []
pair_list = pd.DataFrame(columns = ['Exchange','Crypto','Fiat'])
for ex in ex_list:
    cur_exchange = ccc_df.loc[ex].dropna()
    ex_currencies = cur_exchange.index
    all_crypto = list(set(all_crypto))
    for crypto in ex_currencies:
        fiat_list = cur_exchange.loc[crypto]
        for fiat in fiat_list:
            #Make a List of Cryptos to go through
            pair_list=pair_list.append(pd.DataFrame([[ex,crypto,fiat]],columns = ['Exchange','Crypto','Fiat']))
        
## Add cccagg USD exchange rate for all cryptos 
cccagg_df = pd.DataFrame(columns = ['Exchange','Crypto','Fiat'])
cccagg_df['Crypto']=pair_list['Crypto'].unique()
cccagg_df['Exchange']='cccagg'
cccagg_df['Fiat']='USD'
pair_list=pair_list.append(cccagg_df)
pair_list.reset_index(inplace=True,drop=True)
pair_list.to_csv(os.path.join(Data_Path,"Exchange_Pair_List.csv"))
pair_list

,Exchange,Crypto,Fiat
0,ABCC,CND,BTC
1,ABCC,CND,ETH
2,ABCC,ELF,ETH
3,ABCC,ELF,BTC
4,ABCC,BEE,BTC
...,...,...,...
36842,cccagg,XMRG,USD
36843,cccagg,GGS,USD
36844,cccagg,DUBI,USD
36845,cccagg,GUNS,USD


In [3]:
## Download all coins from CCC

#This has been outsourced to an external python script

def unix_time(d):
    return calendar.timegm(d.timetuple())
end_date = datetime.today()

# os.remove(os.path.join(Data_Path,"CCC_new.db"))
conn = sqlite3.connect(os.path.join(Data_Path,"CCC_new.db"))

# cursor = conn.cursor()
all_crypto = []

#Benchmark
pair_list = pd.read_csv(os.path.join(Data_Path,"Exchange_Pair_List.csv"))

def partition(pair_list,threads=4):
    np.array_split(range(len(pair_list)),threads)
    return np.array_split(range(len(pair_list)),threads)

def download_rows(pair_list,res_index=0,start=0,end=0,sleep_time=60):
    start_time = time.time()
    res_df = pd.DataFrame()
    cur_sleep_time = sleep_time
    if not end:
        end = len(pair_list)
    for index,row in pair_list[start:end].iterrows():
        crypto = row['Crypto']
        fiat = row['Fiat']
        ex = row['Exchange']
        try:
            hit_url = 'https://min-api.cryptocompare.com/data/histoday?fsym='+str(crypto)+'&tsym='+str(fiat)+'&limit=2000&aggregate=1&toTs='+str(unix_time(end_date))+'&e='+ str(ex)
            #Check for rate limit! If we hit rate limit, then wait!
            while True:
                d = json.loads(requests.get(hit_url).text)
                if d['Response'] =='Success':
                    df = pd.DataFrame(d["Data"])
                    if index%1000==0:
                        print('hitting', ex, crypto.encode("utf-8"), fiat, 'on thread', res_index) 
                    if not df.empty:
                        df['Source']=ex
                        df['From']=crypto
                        df['To']=fiat
                        df=df[df['volumeto']>0.0]
                        res_df = res_df.append(df)
                    cur_sleep_time = sleep_time
                    break
                else:
                    cur_sleep_time = int((np.random.rand()+.5)*cur_sleep_time*1.5)
                    if cur_sleep_time>1800:
                        print('Hit rate limit on thread %d, waiting for %ds'%(res_index,cur_sleep_time))
                    time.sleep(cur_sleep_time)
                
        except Exception as err:
            time.sleep(15)
            print('problem with',ex.encode("utf-8"),crypto,fiat)
    end_time = time.time()
    result_dfs[res_index] = res_df
    print('Total time spent %ds on thread %d'%(end_time-start_time,res_index))
    

threads = 4
parts = partition(pair_list[:500],threads)
thread_list = [0 for _ in range(threads)]
result_dfs = [0 for _ in range(threads)]

for i, pair in enumerate(parts):
    thread_list[i] = Thread(target=download_rows, args=(pair_list,i,pair[0],pair[-1],))
for i in range(threads):
    # starting thread i 
    thread_list[i].start() 
for i in range(threads):
    thread_list[i].join() 
for result in result_dfs:
    result.to_sql("Data", conn, if_exists="append")
    print(len(result))
conn.commit()
conn.close()

hitting ABCC b'CND' BTC on thread 0
Total time spent 115s on thread 0
Total time spent 121s on thread 2
Total time spent 122s on thread 3
Total time spent 122s on thread 1
1636
10994
14027
24889


In [2]:
hit_url = 'https://min-api.cryptocompare.com/data/histoday?fsym=LTC&tsym=BTC&limit=2000&aggregate=1&toTs=1570012197&e=Bitfinex'
d = json.loads(requests.get(hit_url).text)
#Check for rate limit! If we hit rate limit, then wait!
df = pd.DataFrame(d["Data"])
print(d['Response'])
print(d)

Success
{'Response': 'Success', 'Type': 100, 'Aggregated': False, 'Data': [{'time': 1397174400, 'close': 0, 'high': 0, 'low': 0, 'open': 0, 'volumefrom': 0, 'volumeto': 0}, {'time': 1397260800, 'close': 0, 'high': 0, 'low': 0, 'open': 0, 'volumefrom': 0, 'volumeto': 0}, {'time': 1397347200, 'close': 0, 'high': 0, 'low': 0, 'open': 0, 'volumefrom': 0, 'volumeto': 0}, {'time': 1397433600, 'close': 0, 'high': 0, 'low': 0, 'open': 0, 'volumefrom': 0, 'volumeto': 0}, {'time': 1397520000, 'close': 0, 'high': 0, 'low': 0, 'open': 0, 'volumefrom': 0, 'volumeto': 0}, {'time': 1397606400, 'close': 0, 'high': 0, 'low': 0, 'open': 0, 'volumefrom': 0, 'volumeto': 0}, {'time': 1397692800, 'close': 0, 'high': 0, 'low': 0, 'open': 0, 'volumefrom': 0, 'volumeto': 0}, {'time': 1397779200, 'close': 0, 'high': 0, 'low': 0, 'open': 0, 'volumefrom': 0, 'volumeto': 0}, {'time': 1397865600, 'close': 0, 'high': 0, 'low': 0, 'open': 0, 'volumefrom': 0, 'volumeto': 0}, {'time': 1397952000, 'close': 0, 'high': 0,

In [7]:
(np.random.rand()+.5)

0.6505732222255493

In [14]:
print('Hit rate limit, waiting for %ds'%time.sleep(30))
print('dine')

TypeError: %d format: a number is required, not NoneType

In [ ]:
df = pd.DataFrame({
  'first column': [1, 2, 3, 4],
  'second column': [10, 20, 30, 40]
})

res_df = pd.DataFrame()
res_df= res_df.append(df)
res_df

In [ ]:
pair_list[10:30]

In [ ]:
list(range(100))[10:]

In [ ]:
## Download all coins from CCC
from threading import Thread 
import sqlite3
import time
def unix_time(d):
    return calendar.timegm(d.timetuple())
end_date = datetime.today()

conn = sqlite3.connect(os.path.join(Data_Path,"CCC_new.db"))

cursor = conn.cursor()
all_crypto = []

#Benchmark
pair_list = pd.read_csv(os.path.join(Data_Path,"Exchange_Pair_List.csv"))
pair_list = pair_list.iloc[:100]
start = time.time()
res_df = pd.DataFrame()

def download_rows(pair_list,conn,start=0,end=0):
    if not end:
        end = len(pair_list)
    for index,row in pair_list[start:end].iterrows():
        crypto = row['Crypto']
        fiat = row['Fiat']
        ex = row['Exchange']

#         try:
        hit_url = 'https://min-api.cryptocompare.com/data/histoday?fsym='+str(crypto)+'&tsym='+str(fiat)+'&limit=2000&aggregate=1&toTs='+str(unix_time(end_date))+'&e='+ str(ex)
        d = json.loads(requests.get(hit_url).text)
        df=pd.DataFrame(d["Data"])
        if not df.empty:
            print('hitting',ex,crypto.encode("utf-8"),fiat) 
            all_crypto = all_crypto + [crypto]
            df['Source']=ex
            df['From']=crypto
            df['To']=fiat
            df=df[df['volumeto']>0.0]
            if index%1000 == 1:
                res_df.to_sql("Data", conn, if_exists="append")
                res_df = pd.DataFrame()
            else:
                res_df = res_df.append(df)
#         except Exception as err:
#             time.sleep(15)
#             print('problem with',ex.encode("utf-8"),crypto)
    res_df.to_sql("Data", conn, if_exists="append")
    res_df = pd.DataFrame()
    end = time.time()
    print('Total time spent %ds'%(end-start))
    
t1 = Thread(target=download_rows, args=(pair_list,conn,0,50,))
t2 = Thread(target=download_rows, args=(pair_list,conn,50,100,))
# starting thread 1 
t1.start() 
# starting thread 2 
t2.start() 

# wait until thread 1 is completely executed 
t1.join() 
# wait until thread 2 is completely executed 
t2.join() 

# url='https://min-api.cryptocompare.com/data/all/exchanges'
# data = requests.get(url).json()
# ccc_df = pd.DataFrame.from_dict(data).T

# ex_list = list(ccc_df.index)
# ex_list.remove('EtherDelta')

# conn = sqlite3.connect(os.path.join(Data_Path,"CCC_new.db"))

# cursor = conn.cursor()

# all_crypto = []
# for ex in ex_list:
#     cur_exchange = ccc_df.loc[ex].dropna()
#     ex_currencies = cur_exchange.index
#     all_crypto = list(set(all_crypto))
#     for crypto in ex_currencies:
#         fiat_list = cur_exchange.loc[crypto]
#         for fiat in fiat_list:
#             try:
#                 hit_url = 'https://min-api.cryptocompare.com/data/histoday?fsym='+str(crypto)+'&tsym='+str(fiat)+'&limit=2000&aggregate=1&toTs='+str(unix_time(end_date))+'&e='+ str(ex)
# #                 print(hit_url)
#                 d = json.loads(requests.get(hit_url).text)
#                 df=pd.DataFrame(d["Data"])
#                 if not df.empty:
# #                     print('hitting',ex,crypto.encode("utf-8"),fiat) 
#                     all_crypto = all_crypto + [crypto]
#                     df['Source']=ex
#                     df['From']=crypto
#                     df['To']=fiat
#                     df=df[df['volumeto']>0.0]
#                     df.to_sql("Data", conn, if_exists="append")
#             except Exception as err:
#                 time.sleep(15)
#                 print('problem with',ex.encode("utf-8"),crypto)


# #Final run with cccagg
# ex='cccagg'
# all_crypto = list(set(all_crypto))
# fiat_list = ['USD']
# for crypto in all_crypto:
#     for fiat in fiat_list:
#         try:
#             hit_url = 'https://min-api.cryptocompare.com/data/histoday?fsym='+str(crypto)+'&tsym='+str(fiat)+'&limit=2000&aggregate=1&toTs='+str(unix_time(end_date))+'&e='+ str(ex)
# #             print(hit_url)
#             d = json.loads(requests.get(hit_url).text)
#             df=pd.DataFrame(d["Data"])
#             if not df.empty:
#                 print('hitting',ex,crypto,fiat)
#                 df['Source']=ex
#                 df['From']=crypto
#                 df['To']=fiat
#                 df=df[df['volumeto']>0.0]
#                 df.to_sql("Data", conn, if_exists="append")
#         except Exception as err:

#             print('problem with',ex,crypto)

# #Final final run with Etherdelta dropping all weird characters

# ex = 'EtherDelta'
# cur_exchange = ccc_df.loc[ex].dropna()
# ex_currencies = cur_exchange.index
# all_crypto = list(set(all_crypto))
# for crypto in ex_currencies:
#     fiat_list = cur_exchange.loc[crypto]
#     for fiat in fiat_list:
#         try:
#             hit_url = 'https://min-api.cryptocompare.com/data/histoday?fsym='+str(crypto)+'&tsym='+str(fiat)+'&limit=2000&aggregate=1&toTs='+str(unix_time(end_date))+'&e='+ str(ex)
#             d = json.loads(requests.get(hit_url).text)
#             df=pd.DataFrame(d["Data"])
#             if not df.empty:
# #                 print('hitting',ex,crypto.encode("utf-8"),fiat) 
#                 all_crypto = all_crypto + [crypto]
#                 df['Source']=ex
#                 df['From']=crypto
#                 df['To']=fiat
#                 df=df[df['volumeto']>0.0]
#                 df.to_sql("Data", conn, if_exists="append")
#         except Exception as err:
#             time.sleep(10)
#             print('problem with',ex.encode("utf-8"),crypto)
# # Commit changes and close connection
conn.commit()
conn.close()